# Implementing LSA on News Articles using Sklearn

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Import Libraries

In [0]:
import nltk
from nltk.corpus import stopwords
import re
import string
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
import pandas as pd
 
import numpy as np
 
stemmer = PorterStemmer()

In [0]:
import os
from IPython.display import clear_output

In [0]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Download and Preprocess Data
Download data from drive and preprocess by removing stop words, tokenizing and stemming. Store in panda dataframes

In [0]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [0]:
updated_stopwords = stopwords.words('english')
updated_stopwords.remove('he')
updated_stopwords.remove('him')
updated_stopwords.remove('his')
updated_stopwords.remove('himself')
updated_stopwords.remove('she')
updated_stopwords.remove("she's")
updated_stopwords.remove('her')
updated_stopwords.remove('hers')
updated_stopwords.remove('herself')
updated_stopwords.append('said')
updated_stopwords.append('also')
updated_stopwords.append('would')

In [0]:
def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')

In [0]:
row_list = []

def load_data (Newspaper):
  global row_list
  male_pronouns = ['he','him','his','himself']
  female_pronouns = ['she','her','hers','herself',"her's",]
  
  count = 0
  row_list = []
  date_list = [date for date in os.listdir(base_dir + Newspaper)]
  date_list = date_list[::-1]
 
  for date in date_list:
    for section in os.listdir(base_dir + Newspaper + '/' + date):
      for article in os.listdir(base_dir + Newspaper + '/' + date + '/' + section):
        if("sport" in section.lower()):

          if (article[-4:] == '.txt'):

            count = count + 1
            clear_output(wait=True)
            title=" "
            gender = "Neither"
            print('Article Count: ', count)
            if(count == 10000):
              return
            with open(base_dir + Newspaper + '/' + date + '/' + section + '/' +  article, 'r') as handler:
              title = handler.readline()
              body = handler.read()
              if (any( contains_word(body,x) for x in male_pronouns)):
                gender = "Male"
              if (any( contains_word(body,x) for x in female_pronouns)):
                if gender == "Male":
                  gender = "Female/Male"
                else:
                  gender = "Female"

            row = {'Date': date,"Title":title[:-2] ,'Section': section, 'Newspaper': Newspaper, 'Body': body,'Gender':gender}
            row_list.append(row)

In [0]:
load_data('The_News_International')

Article Count:  10000


In [0]:
List_The_News_International = row_list
df_The_News_International = pd.DataFrame(List_The_News_International)
print('Total Rows: ', len(df_The_News_International.index))
df_The_News_International.tail()

Total Rows:  9999


,Body,Date,Gender,Newspaper,Section,Title
9994,LAHORE: Peshawar Zalmi’s wicketkeeper batsman ...,2019_02_17,Male,The_News_International,21_SportsNews,Kamran’s focus is on his game not 1000 PSL runs
9995,Dubai: Multan Sultans opened their account in ...,2019_02_17,Male,The_News_International,21_SportsNews,HBL PSL-4: Afridi’s brilliance too much for Is...
9996,SYDNEY: Superstar mare Winx easily won her 30t...,2019_02_17,Female/Male,The_News_International,22_SportsNews,Record-breaking mare Winx wins 30th race
9997,LONDON: Brighton moved into the FA Cup quarter...,2019_02_17,Male,The_News_International,22_SportsNews,Brighton edge into FA Cup quarters
9998,"PROGRESSO, Argentina: The funeral of Cardiff C...",2019_02_17,Male,The_News_International,22_SportsNews,Sala’s funeral takesplace in Argentina


In [0]:
print("Male\n", df_The_News_International.loc[df_The_News_International['Gender'] == 'Male'].count())
print("\nFemale\n", df_The_News_International.loc[df_The_News_International['Gender'] == 'Female'].count())

# male_count = df_The_News_International['Gender'=='Male'].count()
# female_count = df_The_News_International['Gender'=='Female'].count()
# print("male count=",male_count)
# print("female count=",female_count)

# df_The_News_International['Gender'=='Female'].count()

Male
 Body         5849
Date         5849
Gender       5849
Newspaper    5849
Section      5849
Title        5849
dtype: int64

Female
 Body         668
Date         668
Gender       668
Newspaper    668
Section      668
Title        668
dtype: int64


In [0]:
print(df_The_News_International.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 6 columns):
Body         9999 non-null object
Date         9999 non-null object
Gender       9999 non-null object
Newspaper    9999 non-null object
Section      9999 non-null object
Title        9999 non-null object
dtypes: object(6)
memory usage: 468.8+ KB
None


In [0]:
df_The_News_International['Date'] =  pd.to_datetime(df_The_News_International['Date'], format='%Y_%m_%d')

In [0]:
import re

def uniform_sections(name):
  pattern = '1|2|3|4|5|6|7|8|9|0|_| |Page|News'
  result = re.sub(pattern, '', name)

  return result

In [0]:
df_The_News_International['Section'] = df_The_News_International['Section'].apply(lambda x: uniform_sections(x))
df_The_News_International.head()

,Body,Date,Gender,Newspaper,Section,Title
0,LAHORE: The Pakistan Wrestling Federation (PW...,2018-02-01,Male,The_News_International,Sports,PWF names wrestlers for Commonwealth Games
1,DUBAI: The International Cricket Council (ICC)...,2018-02-01,Neither,The_News_International,Sports,ICC opens corruption probe against Ajman League
2,MOSCOW: President Vladimir Putin told Russian ...,2018-02-01,Male,The_News_International,Sports,Putin sorry for not ‘shielding’ Russia athletes
3,LONDON: Amir Khan threw water over opponent Ph...,2018-02-01,Male,The_News_International,Sports,Amir throws water at rival over personal life ...
4,LAHORE: Pakistan Cricket Board (PCB) Wednesday...,2018-02-01,Male,The_News_International,Sports,Khalid’s appeal against spot-fixing ban rejected


## Transform with TF-IDF

In [0]:
transformer = TfidfVectorizer()
tfidf = transformer.fit_transform(df_The_News_International.Body)
# print(transformer)

In [0]:
transformer.get_feature_names()

['00',
 '000',
 '000km',
 '000m',
 '000th',
 '000tpa',
 '001',
 '001st',
 '004',
 '007',
 '0072nd',
 '007race',
 '00am',
 '00junaid',
 '00pm',
 '00pts',
 '00sec',
 '00shaheen',
 '00shan',
 '01',
 '010',
 '0104',
 '011',
 '0111712',
 '0120',
 '013sec',
 '014sec',
 '015sec',
 '018',
 '019',
 '01975',
 '01982',
 '01994',
 '01m',
 '01sec',
 '02',
 '0200',
 '0206',
 '020th',
 '021',
 '023',
 '024',
 '024sec',
 '026',
 '029',
 '02balochistan',
 '02m',
 '02s',
 '03',
 '030',
 '03003870116',
 '03006412109',
 '03009257377',
 '0301',
 '03070130015',
 '031',
 '0316',
 '0321',
 '03226593247',
 '03249822058',
 '03335589876',
 '034',
 '03412269283',
 '0342',
 '035',
 '035sec',
 '039',
 '03s',
 '03sec',
 '04',
 '040',
 '0400',
 '04013',
 '041',
 '044',
 '04400m',
 '044sec',
 '045',
 '048',
 '04km',
 '04sec',
 '04th',
 '05',
 '051',
 '053',
 '057',
 '05navy',
 '05s',
 '05sec',
 '05secs',
 '06',
 '060',
 '0600',
 '063',
 '0630',
 '0655gmt',
 '067',
 '067sec',
 '068',
 '06m',
 '06sec',
 '07',
 '070',
 '

## Singular Value Decomposition
We now perform the SVD computation to extract the feature representations of the words. The thing to keep in mind is that the TruncatedSVD model returns the components for the rows alone. Therefore, if the rows of the tf-idf matrix represent the documents, then the TruncatedSVD model will return the feature vectors of the documents. To retrieve the feature vectors of the words, we can simply fit the transpose of the tf-idf matrix to the model.

Here we are interested in finding term-term similarity and hence use the transpose of the tf-idf matrix as the input to the SVD model.

In [0]:
svd = TruncatedSVD(n_components = 300)
lsa = svd.fit_transform(tfidf.T)

## Term-Term Similarity
We now explore the LSA model to compute term-term similarity.

We define two functions for this purpose: 
1.   **getClosestTerm** returns the term in the corpus that is most similar to the queried term.

2.   **kClosestTerms** returns k terms that are closest to the queried term.



In [0]:
def getClosestTerm(term,transformer,model):
 
    term = stemmer.stem(term)
    index = transformer.vocabulary_[term]      
 
    model = np.dot(model,model.T)
    searchSpace =np.concatenate( (model[index][:index] , model[index][(index+1):]) )  
 
    out = np.argmax(searchSpace)
 
    if out<index:
        return transformer.get_feature_names()[out]
    else:
        return transformer.get_feature_names()[(out+1)]

In [0]:
def kClosestTerms(k,term,transformer,model):
 
    term = stemmer.stem(term)
    index = transformer.vocabulary_[term]
 
    model = np.dot(model,model.T)
 
    closestTerms = {}
    for i in range(len(model)):
        closestTerms[transformer.get_feature_names()[i]] = model[index][i]
 
    sortedList = sorted(closestTerms , key= lambda l : closestTerms[l])
 
    return sortedList[::-1][0:k]

## Sample Outputs

In [0]:
kClosestTerms(20,"he",transformer,lsa)


In [0]:
kClosestTerms(20,"she",transformer,lsa)


In [0]:
kClosestTerms(20,"her",transformer,lsa)

In [0]:
kClosestTerms(20,"him",transformer,lsa)


In [0]:
kClosestTerms(20,"woman",transformer,lsa)

In [0]:
kClosestTerms(20,"man",transformer,lsa)


NameError: ignored

## Pepsi Cola..I need result and sleep ##

In [0]:
!pip install pyLDAvis

In [0]:

#modules to build the topic extracting models
from gensim import corpora, models
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
import re
import string
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn import metrics

import os
from IPython.display import clear_output

stemmer = PorterStemmer()

pd.set_option("display.max_colwidth", 200)

## whatever

In [0]:
documents = []
for c in row_list:
  documents.append(c['Body'])

In [0]:
#creating the dictionary
dictionary = corpora.Dictionary([" ".join(documents).split()]) 
print('{} different terms in the corpus'.format(len(dictionary)))
#creating the bag of words object
bow_corpus = [dictionary.doc2bow(text.split()) for text in documents]

21070 different terms in the corpus


In [0]:
tfidf_model = models.TfidfModel(bow_corpus) # creating the tf-idf model
tfidf_corpus = tfidf_model[bow_corpus]

In [0]:

total_topics = 50
lda_model_tfidf = models.LdaModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=total_topics,passes=1, random_state=47)
lda_model_bow = models.LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=total_topics,passes=1, random_state=47)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [0]:
minister_LDAvis = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)
pyLDAvis.save_html(minister_LDAvis, 'lda_mna+mpa.html')
minister_LDAvis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4     -0.015653 -0.001629       1        1  5.654246
49     0.007048 -0.008986       2        1  5.193263
13    -0.043612  0.014965       3        1  5.170354
46     0.009245  0.004724       4        1  4.967786
41    -0.023799 -0.005708       5        1  4.895751
36    -0.039947 -0.010279       6        1  4.331651
40    -0.008338 -0.007967       7        1  4.153944
31     0.000134  0.001069       8        1  3.938169
1     -0.017505 -0.021466       9        1  3.527248
19    -0.003924  0.022055      10        1  3.517950
8     -0.020508  0.019219      11        1  3.003902
38    -0.012356 -0.004355      12        1  2.782142
21    -0.008858 -0.000455      13        1  2.699497
43    -0.017447  0.034663      14        1  2.637129
3      0.003658 -0.020974      15        1  2.545896
18    -0.024963  0.018696      16        1  2.456235
33    -0.000681 -0.004790      17        1  2.121141
15     0.013394 -0.020646      18        1  1.971567
37     0.013253 -0.017505      19        1  1.911276
25    -0.018828 -0.010982      20        1  1.894141
48    -0.025752 -0.006085      21        1  1.887977
42    -0.000789 -0.004571      22        1  1.831793
5     -0.028851 -0.024736      23        1  1.830145
44    -0.003706  0.003051      24        1  1.713343
47    -0.004764  0.003286      25        1  1.574393
45    -0.019577  0.019844      26        1  1.546221
22     0.003026  0.024115      27        1  1.541183
39    -0.016160  0.029486      28        1  1.345724
7      0.014607 -0.043040      29        1  1.302257
35    -0.002229 -0.007098      30        1  1.282567
10    -0.017147 -0.005162      31        1  1.269044
11    -0.013049 -0.000679      32        1  1.181272
9     -0.017291 -0.013549      33        1  1.179031
2      0.010968  0.019063      34        1  1.052370
27     0.071086  0.019686      35        1  1.052139
16     0.016510 -0.016470      36        1  1.032921
32    -0.010034  0.011692      37        1  0.992406
29    -0.005230 -0.009238      38        1  0.943277
34     0.009610  0.005954      39        1  0.934203
12     0.024016 -0.019764      40        1  0.871182
14     0.022858  0.013589      41        1  0.859769
24     0.019938 -0.031657      42        1  0.745831
20    -0.002887 -0.018177      43        1  0.543661
17     0.014638  0.023334      44        1  0.466735
28    -0.003270  0.010932      45        1  0.389696
23     0.004898  0.010939      46        1  0.370640
6      0.024493  0.030874      47        1  0.338743
26     0.049891  0.015954      48        1  0.272506
0      0.052195  0.013780      49        1  0.146973
30     0.041688 -0.034999      50        1  0.128713, topic_info=      Category         Freq      Term        Total  loglift  logprob
7837   Default  5686.000000        he  5686.000000  30.0000  30.0000
11464  Default  4077.000000    minist  4077.000000  29.0000  29.0000
21065  Default  4589.000000         ’  4589.000000  28.0000  28.0000
7351   Default  4766.000000    govern  4766.000000  27.0000  27.0000
7977   Default  2530.000000        hi  2530.000000  26.0000  26.0000
13261  Default  3250.000000  pakistan  3250.000000  25.0000  25.0000
2084   Default  2048.000000   billion  2048.000000  24.0000  24.0000
21066  Default  3268.000000         “  3268.000000  23.0000  23.0000
21067  Default  3251.000000         ”  3251.000000  22.0000  22.0000
12184  Default  2086.000000    nation  2086.000000  21.0000  21.0000
17883  Default  1286.000000     state  1286.000000  20.0000  20.0000
14417  Default  1881.000000     prime  1881.000000  19.0000  19.0000
18589  Default  1415.000000       tax  1415.000000  18.0000  18.0000
9809   Default  1472.000000      khan  1472.000000  17.0000  17.0000
13434  Default  1249.000000     parti  1249.000000  16.0000  16.0000
3956   Default  1778.000000   countri  1778.000000  15.0000  15.0000
12059  Default   920.000000 

In [0]:
from google.colab import files
files.download('lda_mna+mpa.html')